In [1]:
import sys, os
import pandas as pd
import numpy as np

In [2]:
trainingDataFilename = 'trainingSet.csv'
testDataFilename = 'testSet.csv'
modelIdx = 2

In [3]:
def sigmoid(scores):
    predictions = np.zeros(len(scores))
    for i in range(len(predictions)):
        if scores[i] >= 0:
            predictions[i] +=  1.0 / (1.0 + np.exp(-scores[i]))
        else:
            predictions[i] += np.exp(scores[i]) / (1.0 + np.exp(scores[i]))
    return predictions

def lr(trainingSet, testSet):
    print len(trainingSet.columns)
    regularization = 0.01
    step_size = 0.01
    
    max_iterations = 500
    tol = 1e-6
    
    count = 0
    
    train_labels = trainingSet['decision']    
    trainingSet = trainingSet.drop('decision', axis=1)
    
    #print train_labels, trainingSet
    w = np.zeros(len(trainingSet.columns) + 1)
    
    # Add intercept
    X = np.array(trainingSet)
    Y = np.array(train_labels)
    intercept = np.ones((X.shape[0], 1))
    #X = np.concatenate((X, intercept.T), axis=1)
    X = np.hstack((X, intercept))
    diff = 100.0
    
    while(count < max_iterations and diff > tol):
        count += 1
        norm_old = np.linalg.norm(w)
        
        scores = np.dot(X, w)
        predictions = sigmoid(scores)

        gradient = np.dot(X.T, (predictions - Y))

        for j in range(len(w)):
            gradient[j] += regularization * w[j]
            
        #gradient /= len(train_labels)
        w -= step_size * gradient
        norm_new = np.linalg.norm(w)
        
        diff = abs(norm_new - norm_old)
        #print w
        print count, diff
    
    return w

In [4]:
def svm(trainingSet, testSet):
    #print len(trainingSet.columns)
    regularization = 0.01
    step_size = 0.50
    
    max_iterations = 500
    tol = 1e-6
    #print len(trainingSet[trainingSet['decision'] == 1])
    count = 0
    train_labels = trainingSet['decision']    
    trainingSet = trainingSet.drop('decision', axis=1)

    w = np.zeros(len(trainingSet.columns) + 1)
    
    # Add intercept
    X = np.array(trainingSet)
    Y = np.array(train_labels)
    #print train_labels
    for i in range(len(Y)):
        if Y[i] == 0:
            Y[i] = -1.0
        else:
            Y[i] = 1.0
    #print Y.tolist()
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((intercept, X))
    diff = 100.0
    while(count < max_iterations and diff > tol):
        count += 1
        norm_old = np.linalg.norm(w)
        
        predictions = np.dot(X, w)
        #print X.shape, w.shape,predictions.shape
#         p = []
#         for i in range(0, 5200):
#             p.append(sum([w[j] * X[i][j] for j in range(len(trainingSet.columns))]))
        
#         print predictions, p 
        #print len(predictions)
        #print np.dot(w, X[0]), predictions[0]     
        error = 0
        gradient = np.zeros(len(w))
        for i in range(len(predictions)):
            if predictions[i] * Y[i] < 1.0:
                error += 1
                #gradient -= 1.0 * Y[i] * X[i]
                gradient -= np.multiply(X[i], Y[i])
            
        gradient /= 1.0 * len(train_labels)
        #print gradient.shape, X[0].shape
        
        for j in range(1, len(gradient)):
            gradient[j] += 1.0 * regularization * w[j]

        w -= 1.0 * step_size * gradient
        norm_new = np.linalg.norm(w)
        diff = abs(norm_new - norm_old)
        print count, diff, error
    #print w
    return w

In [5]:
def get_accuracy_lr(w, trainingSet, testSet):
    total_train = len(trainingSet)
    count_train = 0
    total_test = len(testSet)
    count_test = 0
    
    train_labels = trainingSet['decision']
    test_labels = testSet['decision']
    
    trainingSet = trainingSet.drop('decision', axis=1)
    testSet = testSet.drop('decision', axis=1)
    
    # Training accuracy
    X = np.array(trainingSet)
    Y = np.array(train_labels)
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((X, intercept))
    
    scores = np.dot(X, w)
    predictions = sigmoid(scores)
        
    for i in range(len(predictions)):
        if predictions[i] > 0.5:
            predictions[i] = 1
        else:
            predictions[i] = 0

    for i in range(len(predictions)):    
        if predictions[i] == int(train_labels[i]):
            count_train += 1

    training_accuracy = 1.0 * count_train/total_train
    print 'Training Accuracy LR:', '%.2f' % training_accuracy, count_train, total_train
    
    # Test accuracy
    X = np.array(testSet)
    Y = np.array(test_labels)
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((X, intercept))

    scores = np.dot(X, w)
    predictions = sigmoid(scores)

    for i in range(len(predictions)):
        if predictions[i] > 0.5:
            predictions[i] = 1
        else:
            predictions[i] = 0

    for i in range(len(predictions)):    
        if predictions[i] == int(test_labels[i]):
            count_test += 1
            
    test_accuracy = 1.0 * count_test/total_test
    print 'Test Accuracy LR:', '%.2f' % test_accuracy, count_test, total_test
    
def get_accuracy_svm(w, trainingSet, testSet):
    total_train = len(trainingSet)
    count_train = 0
    total_test = len(testSet)
    count_test = 0
    
    train_labels = trainingSet['decision']
    test_labels = testSet['decision']
    
    trainingSet = trainingSet.drop('decision', axis=1)
    testSet = testSet.drop('decision', axis=1)
    
    # Training accuracy
    X = np.array(trainingSet)
    Y = np.array(train_labels)
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((intercept, X))
    
    predictions = np.dot(X, w)
    
    for i in range(len(predictions)):
        if predictions[i] > 0.0:
            predictions[i] = 1
        else:
            predictions[i] = 0

    for i in range(len(predictions)):    
        if predictions[i] == int(Y[i]):
            count_train += 1

    training_accuracy = 1.0 * count_train/total_train
    print 'Training Accuracy SVM:', '%.2f' % training_accuracy
    
    # Test accuracy
    X = np.array(testSet)
    Y = np.array(test_labels)
    intercept = np.ones((X.shape[0], 1))
    X = np.hstack((intercept, X))

    predictions = np.dot(X, w)

    for i in range(len(predictions)):
        if predictions[i] > 0.0:
            predictions[i] = 1
        else:
            predictions[i] = 0

    for i in range(len(predictions)):    
        if predictions[i] == int(Y[i]):
            count_test += 1
            
    test_accuracy = 1.0 * count_test/total_test
    print 'Test Accuracy SVM:', '%.2f' % test_accuracy

In [6]:
trainingSet = pd.read_csv(trainingDataFilename)
testSet = pd.read_csv(testDataFilename)

if modelIdx == 1:
    w = lr(trainingSet, testSet)
    get_accuracy_lr(w, trainingSet, testSet)
else:
    w = svm(trainingSet, testSet)
    get_accuracy_svm(w, trainingSet, testSet)

1 3.339013477533335 5200
2 5.102754375694117 2261
3 1.7424639012953653 2939
4 1.082706474824259 2261
5 4.409507422907081 2939
6 6.2512039712400975 2261
7 9.54483690793526 2939
8 8.99377302443336 2261
9 6.952230114911771 2261
10 4.536711086625103 2939
11 2.206644962245922 2261
12 0.7626290062937056 2939
13 2.25614755760465 2261
14 5.375156084574075 2939
15 5.546985262636933 2261
16 6.768573716125723 2537
17 6.213986063382413 2261
18 1.474639274023926 1613
19 3.277340751504049 2939
20 3.7568138378250318 2261
21 6.719150773238768 2925
22 6.0475515157692445 2261
23 4.0800176320092625 2105
24 2.1444537006715407 2939
25 0.8416070412852186 2261
26 1.6013577850836427 2939
27 2.0779656339494075 2261
28 4.7761526756133605 2934
29 4.363502332903405 2261
30 0.2056918274983488 1427
31 3.2713164940234734 2103
32 1.495677706804031 2939
33 0.5396242570613481 2261
34 1.7099495347727682 2939
35 1.9301067780512078 2261
36 4.366583501566485 2912
37 3.8729286268549306 2261
38 0.2731474026424614 1378
39 0.6

311 0.4337427203373849 2316
312 0.35539838607522967 2248
313 0.4304974013910723 2315
314 0.3529311525376997 2248
315 0.42970936490845446 2317
316 0.35290697088093026 2248
317 0.4241720112828631 2313
318 0.34942369022218145 2247
319 0.42474137469091033 2317
320 0.35072390267562525 2247
321 0.4168207313778254 2310
322 0.34354952287066 2247
323 0.41861807763817893 2318
324 0.3460598551027658 2247
325 0.41214146173413013 2311
326 0.34031380708525916 2247
327 0.4139838509433673 2318
328 0.34285629494645065 2247
329 0.40757678356588656 2311
330 0.33716617441782404 2247
331 0.40946457183130036 2318
332 0.3397396825081742 2247
333 0.4019447133992031 2310
334 0.3365300712277133 2244
335 0.4050739460946531 2316
336 0.3402908586113469 2244
337 0.3998653074397467 2309
338 0.3357442176098573 2244
339 0.39474523741770895 2309
340 0.33493275723660787 2241
341 0.39335209436358554 2309
342 0.3353819093060082 2240
343 0.39322317763966197 2309
344 0.335824795287472 2240
345 0.39165797578858985 2308
346 0